<a href="https://colab.research.google.com/github/vansjyo/DeepMars-Colab/blob/master/Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import keras
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
from PIL import Image
import gdal
import cv2
from skimage import io
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

In [0]:
train_images = sorted(glob("/content/drive/My Drive/DeepCrater/Test/*tif"))
label_images = sorted(glob("/content/drive/My Drive/DeepCrater/Labels/*tif"))
print("Labelled : ", len(label_images), "Training : ", len(train_images))

def ImportImage( filename):
    img = io.imread(filename)
    img = img.astype('float32') / 255
    print(img.shape)
    return np.array(img)[:]

X = np.array( [ ImportImage(img) for img in train_images ] )
Y = np.array( [ ImportImage(img) for img in label_images ] )
plt.imshow(X[0])

In [0]:
augmented_x = []
augmented_y = []

# Augment rare
datagen = ImageDataGenerator(
    rotation_range=90,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2, # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1, # randomly shift images vertically (fraction of total height)
    zoom_range=[0.96,1],  # set range for random zoom
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True,  # randomly flip images
    )
datagen.fit(X)

iteration = 0
limit = 20

for x_batch, y_batch in datagen.flow(X, Y, batch_size = 1):
    fname_x = "/content/drive/My Drive/DeepCrater/Augmented//Train/Train_" + str(iteration) + ".tif"
    fname_y = "/content/drive/My Drive/DeepCrater/Augmented/Label/Label_" + str(iteration) + ".tif"
    # io.imsave(fname_x, x_batch)
    # io.imsave(fname_y, y_batch)
    augmented_x.extend(x_batch[:])   
    augmented_y.extend(y_batch[:])    
    iteration += 1
    if iteration >= limit:
        break

augmented_x = np.array(augmented_x)
augmented_y = np.array(augmented_y)
print(augmented_x.shape)

In [0]:
fig,a =  plt.subplots(10,2)
for i in range(10):
  a[i][0].imshow(augmented_x[i])
  a[i][1].imshow(augmented_y[i])
plt.figure()

In [0]:
# Shuffle train and test
shufflePermutation = np.random.permutation(len(x_train))
x_train = x_train[shufflePermutation]
y_train = y_train[shufflePermutation]